In [1]:
import wandb
from ultralytics import YOLO
import cv2
import pandas as pd
import os

In [2]:
!nvidia-smi

Mon Sep  2 01:21:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:00:03.0 Off |                    0 |
| N/A   53C    P8             17W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Login into Weights and Bias

In [4]:
!wandb login

wandb: Currently logged in as: shaunliew20 (shaunliew20-organization). Use `wandb login --relogin` to force relogin


##  Model Training

In [5]:
# Using YOLO's pretrained model architecture and weights for training
model=YOLO('yolov8m-seg.yaml').load('weights/yolov8m-seg.pt')

100%|██████████| 52.4M/52.4M [00:01<00:00, 41.2MB/s]

Transferred 537/537 items from pretrained weights


### Model Training Arguments

In [6]:
args = dict(
    data="/home/jupyter/endoinsight-ai/gcs/config.yaml",  # Using the config_pathfrom GCS
    batch=8,
    device="0",
    epochs=30,
    workers=8,
    optimizer='AdamW',
    save_period=5,
    name='endo_segmentation',
    save=True,
    plots=True,
    patience=5,
    verbose=True,
    iou=0.4,
    conf=0.01,
    degrees=60, # Augmentation arguments
    shear=30,
    perspective=0.0005
)

In [7]:
#Initiating Model Training
results = model.train(**args)

Ultralytics YOLOv8.2.86 🚀 Python-3.10.14 torch-2.4.0+cu121 CUDA:0 (NVIDIA L4, 22478MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.yaml, data=/home/jupyter/endoinsight-ai/gcs/config.yaml, epochs=30, time=None, patience=5, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=None, name=endo_segmentation, exist_ok=False, pretrained=weights/yolov8m-seg.pt, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.01, iou=0.4, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 82.4MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

wandb: Currently logged in as: shaunliew20 (shaunliew20-organization). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

                   all        808       9437      0.913      0.909      0.945      0.783      0.937        0.9      0.943      0.773



30 epochs completed in 2.954 hours.
Optimizer stripped from runs/segment/endo_segmentation/weights/last.pt, 54.8MB
Optimizer stripped from runs/segment/endo_segmentation/weights/best.pt, 54.8MB

Validating runs/segment/endo_segmentation/weights/best.pt...
Ultralytics YOLOv8.2.86 🚀 Python-3.10.14 torch-2.4.0+cu121 CUDA:0 (NVIDIA L4, 22478MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,229,911 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:18<00:00,  2.74it/s]


                   all        808       9437      0.912      0.933      0.946      0.785      0.936      0.909      0.948      0.773
      Black Background        807       1388      0.745      0.901       0.91      0.764      0.874      0.929      0.964      0.816
        Abdominal Wall        714       1286      0.965      0.955      0.979      0.916       0.97      0.952       0.98      0.922
                 Liver        808       2123      0.945       0.96      0.973      0.874      0.958      0.957      0.977      0.876
Gastrointestinal Tract        450        465      0.921      0.955      0.964      0.799      0.947      0.961      0.977      0.808
                   Fat        749       1432      0.918      0.957       0.97      0.865      0.943       0.96      0.978      0.867
               Grasper        608        877      0.973      0.983      0.988       0.92      0.972      0.978      0.985      0.817
     Connective Tissue        168        287      0.894       0.79   

lr/pg0,█▅▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg2,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▁▄▆▆▇▇▇▇▇▇▇▇▇▇▇█▇█▇███████████
metrics/mAP50(M),▁▄▆▇▇▇▇▇▇▇▇███████████████████
metrics/mAP50-95(B),▁▃▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████
metrics/mAP50-95(M),▁▄▅▆▇▆▇▇▇▇▇▇▇▇▇▇▇█████████████
metrics/precision(B),▁▄▅▆▆▇▇▇▇▇▇▇▇█▇▇██▇▇▇█▇█████▇▇
metrics/precision(M),▁▅▅▆▇▇▇▇██▇███▇███████████████
metrics/recall(B),▁▃▅▆▆▆▆▇▇▆▇▇▇▆▇▇▇▇▇▇█▇▇▇▇▇▇███
metrics/recall(M),▁▄▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████▇█▇█████
